 ### 04 - BART - T5
 ----

In [ ]:
import numpy as np
import pandas as pd
import joblib

from transformers import AutoTokenizer, BartModel
import torch
#from transformers import pipeline, set_seed

import pandas as pd
from datasets import load_dataset, load
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import sent_tokenize

In [ ]:
pip install transformers torch

In [ ]:
pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.1 MB/s eta 0:00:00


In [ ]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=fea8bcae2cf486557f0a9374047a324895fc365921056a6ff5bc52ccc8b27d9f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# loading all the sets of data

In [ ]:
import pandas as pd
from datasets import Dataset

# load csv file
dataset_bart_train = pd.read_csv('/content/cleaned_train_data.csv', on_bad_lines='skip', engine='python')
dataset_bart_test = pd.read_csv('/content/cleaned_test_data.csv', on_bad_lines='skip', engine='python')
dataset_bart_validation = pd.read_csv('/content/cleaned_validation_data.csv', on_bad_lines='skip', engine='python')

# convert pandas dataframe to hugging face dataset
dataset_bart_train = Dataset.from_pandas(dataset_bart_train)
dataset_bart_test = Dataset.from_pandas(dataset_bart_test)
dataset_bart_validation = Dataset.from_pandas(dataset_bart_validation)


In [ ]:
# sample of dataset 
dataset_bart_train_s = dataset_bart_train.select(range(3000))
dataset_bart_test_s = dataset_bart_test.select(range(900))
dataset_bart_validation_s = dataset_bart_validation.select(range(900))


In [ ]:
# load the tokenizer and model

from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
'''

def tokenizerr(texts):

    # 'article' column tokenize
    tokenized_inputs = tokenizer(texts['article'], padding='max_length', truncation=True, max_length=512, return_tensors='pt')

    # 'highlights' column tokenize
    tokenized_targets = tokenizer(texts['highlights'], padding='max_length', truncation=True, max_length=60, return_tensors='pt')

    # ignore padding tokens - padding and token ids
    labels = tokenized_targets['input_ids'].clone()
    labels[labels == tokenizer.pad_token_id] = -100

    return {
        'input_ids': tokenized_inputs['input_ids'].tolist(),  # make list - model expects - tokenizer inputs
        'attention_mask': tokenized_inputs['attention_mask'].tolist(),  # make list - model expects  - pay attention/ ignore parts of text
        'labels': labels.tolist()  # make list after adjusting padding tokens - tokenizer labels
    }
'''

In [ ]:
def tokenizerr(examples):

   inputs = [doc for doc in examples['article']]

   model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

   with tokenizer.as_target_tokenizer():

       labels = tokenizer(examples['highlights'], max_length=512, truncation=True, padding="max_length")

   model_inputs["labels"] = labels["input_ids"]

   return model_inputs

tokenizer code from : https://www.analyticsvidhya.com/blog/2024/05/text-summarization-using-googles-t5-base/

In [ ]:
# tokenize both training and test datasets

dataset_bart_train_tok = dataset_bart_train_s.map(tokenizerr, batched=True)
dataset_bart_test_tok = dataset_bart_test_s.map(tokenizerr, batched=True)
dataset_bart_validation_tok = dataset_bart_validation_s.map(tokenizerr, batched=True)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

In [ ]:
print(dataset_bart_train_tok.column_names) # check tokenization was successful


['id', 'article', 'highlights', 'input_ids', 'attention_mask', 'labels']


In [ ]:
from transformers import DataCollatorForSeq2Seq # for nlp tasks processing and formatting

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)  # padding and truncation

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
output_dir = '/content/drive/MyDrive/model_checkpoints/' # checkpoints for model - save to drive

In [ ]:
log_dir = '/content/drive/MyDrive/model_logs/' # save files during training - save to drive


In [ ]:
from transformers import TrainingArguments # perameters for training the model


In [ ]:
#evaluation metrics

In [ ]:
#import rouge score
import evaluate

rouge_score = evaluate.load("rouge")


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode predictions and labels
    predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE scores
    result = rouge_metric.compute(predictions=predictions, references=labels)
    return {
        "rouge1": result["rouge1"].mid.fmeasure,
        "rouge2": result["rouge2"].mid.fmeasure,
        "rougeL": result["rougeL"].mid.fmeasure
    }

    huggingface rouge score: https://huggingface.co/spaces/evaluate-metric/rouge

In [ ]:
# train the model

In [ ]:
from transformers import Trainer, TrainingArguments

# training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_dir=log_dir,
    logging_steps=500,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    gradient_accumulation_steps=6,
    save_total_limit=4,
    load_best_model_at_end=True,
    fp16=True
)

# trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_bart_train_tok,
    eval_dataset=dataset_bart_test_tok,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

# train
trainer.train()

# save
trainer.save_model("/content/drive/MyDrive/fine-tuned-t5-small-summarization")
# Save tokenizer - reproducability
tokenizer.save_pretrained("/content/drive/MyDrive/fine-tuned-t5-small-summarization")

# evaluate
evaluation_results = trainer.evaluate()
print("Evaluation results:", evaluation_results)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 13.42 GiB. GPU 0 has a total capacity of 39.56 GiB of which 13.37 GiB is free. Process 24538 has 26.19 GiB memory in use. Of the allocated memory 15.42 GiB is allocated by PyTorch, and 10.27 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)